# Topic 15: Linear Regression

Linear regression is the most commonly used machine learning algorithm.

The basic concept is quite simple, but there are many variations and extensions upon the ideas, and many implementations for performing linear regression.

We will see a summary of these techniques in this topic.  For those who are interested in data science as a carreer, there is a lot more to learn!

Here is an overview:

* The basic idea

* Definitions and Terms

* Simple Linear Regression

* Building Larger Test Cases

* Polynomial Linear Regressions

* Building Larger Test Cases

* Using Sklearn

* Multiple Linear Regressions (Multiple Inputs)

* Sparse Data

But first, we will set up the Notebook:

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns

In a previous lecture, I said I would explain the purpose of the '%maplotlib inline' line of text.  This is a magic command in Jupyter which links the charts and plots from matplotlib to the Jupyter Notebook, so that the charts appear in the cell outputs.

## The Basic Idea

Suppose we have a sequence of data values, and we would like to determine if there is a simple mathematical relationship between the values.  _For example, the relationship between wind speed and measured wind-chill factor_.

Or suppose the sequence represents the results achieved with application of a given input value. _The example several sources used was number of hours studied as input and the grade received as output_.

Perhaps we would like to then estimate, for a different input value, what output value should be expected?

In the following, we have a set of output values for various input values:

In [ ]:
intro_x = range(0, 10)
intro_y = [-70, -11, -13, 37, -13, 73, 80, 152, 147, 180]
plt.scatter(intro_x, intro_y)
plt.show()

One common approach is to find the _best line_ that "fits" our data:

In [ ]:
intro_z = np.polyfit(intro_x, intro_y, 1)
intro_p = np.poly1d(intro_z)
plt.scatter(intro_x, intro_y)
plt.plot(intro_x, intro_p(intro_x), 'g-')
plt.show()

Using the least-squares fit, the green line provides the best line approximation of the data.  Not all of the points fit exactly on the line (and in fact there is no straight line that exactly holds all of the points).  The distance between any point and the line is the _error_ for that point, and the error for the whole line is the sum of the squares of the errors for every point.  Any other single line that we would draw would have a larger error.

sklearn has a built-in method which computes the _mean squared error_, so we can get a measure of how well the line fits the data.  Many times, we don't really want the mean squared error, but the square root of that value (so that the number uses the same units as the data).

In [ ]:
print(f'{np.sqrt(mean_squared_error(intro_p(intro_x), intro_y)):.2f}')

Using this line, we could estimate the output for any input.  So, for example, if the input was 1.5, the output would be approximately -24 (I just eyeballed this from the graph!).

We could also estimate the output if the input were 10, although this is beyond the range of our input data.  If we look too far beyond the range of our data, that output value may be suspect!  There may be other consideration that would limit the results.  For example, if we were estimating the grade received based on study time spent, we might, if we put in a large enough study time, estimate a grade of 220%, or for the crop example, if we put in a low enough amount of water expect a negative size of crops!  However, using an input value of 10 is close to our data range, so we would expect the prediction to be reasonable.

There is one other point to cover about our plots:  I usually end the cells containing a plot with the command 'plt.show()'.  This is not really required, as you can see here:

In [ ]:
plt.scatter(intro_x, intro_y)
plt.plot(intro_x, intro_p(intro_x), 'g-')

As you can see, the plot still draws.  However, look at that line, '[<matplotlib.lines.Line2D at ...>]'.  What is that all about?  Usually Jupyter prints out the last expression or last result of the cell, then if there was a plot, it prints that.  So we get those ugly lines in our Notebook.  By ending a cell containing a plot with 'plt.show()', this removes that text.

With this introduction, let's define some terms:

## Definitions and Terms

For the simplest linear regression, we are trying to fit the data to a line.

The equation for a line is: $ y = m * x + b $.

In that equation:

* _x_ is an input value,
* _y_ is the associated output value,
* _m_ is the slope of the line, and
* _b_ is the intercept, which is where the line crosses the y-axis (where x == 0).

The goal of simple linear regression is to compute the _m_ and _b_ values given the training set of input values and output values.

For our example above, _m_ = 26.64, and _b_ is -63.69.

While most of the math world calls _m_ the _slope_, in the machine learning community, this is often called the _regression coefficient_.

_x_ is called the _input variable_.  Alternate names are _independent variable_, _predictor variable_, or _feature_.

_y_ is called the _output variable_.  Alternate names are _dependent variable_, _predicted variable_, or _target_.

A _prediction_ is the value that the linear regression model computes for a given input value.  Sometimes we pass an array of input values, and receive an array of the corresponding output values.  In this case _prediction_ refers to the whole array of output values.

An _outlier_ is an unexpected value, a value that is far outside the expected range.  In the following, we add an outlier to the graph.  Since _x_ is a _range_, we can't really add a new element, so I've converted these to a Pandas Series, then concatenated a value which is an outlier:

In [ ]:
# Convert to a pair of Series:
outlier_x = pd.Series(intro_x)
outlier_y = pd.Series(intro_y)
# The outlier.  This must also be a pair of Series
x_add = pd.Series([10])
y_add = pd.Series([12])
# Concatenate the Series, so now we have the outlier with all of the other values
outlier_x = pd.concat([outlier_x, x_add], ignore_index=True)
outlier_y = pd.concat([outlier_y, y_add], ignore_index=True)
# Do the math:
outlier_z = np.polyfit(outlier_x, outlier_y, 1)
# Turn this into an evaluator
outlier_p = np.poly1d(outlier_z)
# Generate the plot
plt.scatter(outlier_x, outlier_y)
plt.plot(outlier_x, outlier_p(outlier_x), 'r-')
plt.plot(intro_x, intro_p(intro_x), 'g-')
plt.show()

The green line was the original solution, the red line shows the solution with this outlier included.  You can see how the one value really skews the line!  Consequently, we can see that with linear regression, it is important that we clean the data, removing any outliers, or samples with missing values, from the dataset.

We can also look at the mean squared error for the red line:

In [ ]:
print(f'{np.sqrt(mean_squared_error(outlier_p(outlier_x), outlier_y)):.2f}')


The general form of the equation for the regression is

$$ y = \Omega_0 + \Omega_1 x^1 + \Omega_2 x^2 ... \Omega_n x^n $$

The _degree_ of the polynomial is the largest exponent of the input variable.  For a line, the degree would be 1, but for a parabola (for example), the degree would be 2, and so on.  The $\Omega$ values are the _coefficients_ of the equation, and any of these may be 0 (except for the coefficient for the largest exponent).

One interesting thing to note: In mathematics, we would normally write the higher-order terms on the left and the lower-order terms on the right.  In data analytics, they reversed this convention!

## Simple Linear Regression

There are several tools in the Anaconda/Miniconda space that can perform linear regression.  Later we will be using _LinearRegression_ from scikit_learn, but for simple linear regressions, many people prefer the solution in Numpy, _polyfit_:

* _polyfit_ is more elegant,
* _polyfit_ is easier to learn, new users find _LinearRegression_ more confusing.
* _polyfit_ seems to be more stable.  _LinearRegression_ changes much more frequently.  If you build a solution using _LinearRegression_, you may find that the code is updated, so your solution is out-of-date.

In a little bit, we will see examples of _LinearRegression_, so you can decide for yourself whether the above points apply to you!

Let's duplicate our earlier example, to explain the steps involved:

In [ ]:
ex1_x = range(0, 10)
ex1_y = [-70, -11, -13, 37, -13, 73, 80, 152, 147, 180]
ex1_z = np.polyfit(ex1_x, ex1_y, 1)
ex1_p = np.poly1d(ex1_z)
plt.scatter(ex1_x, ex1_y)
plt.plot(ex1_x, ex1_p(ex1_x), 'g-')
plt.show()

* The first two lines are our sample data.  Normally, we would fetch (larger) datasets from .csv files, or other sources of data.  For this small example, we simply enter the values as a range and a Python array.  The *ex1_x* values are integers running from 0 through 9 (it stops before 10).  The *ex1_y* values are explicitly listed.

* The third line is the actual machine learning!  The _polyfit_ routine is passed an array of input values, an array of output values, and the degree of the desired polynomial.  In this case, the degree is 1, we are looking for a line.  The return value is an array, giving the coefficients of the polynomial.

In [ ]:
ex1_z

* The fourth line turns the array into an object that computes that polynomial: given an input value, it computes the corresponding output value.

In [ ]:
print(ex1_p)

* The remaining lines generate the plot!

## Building Larger Test Cases

We entered a small test case by hand, and that was useful as an introduction.  But in many cases we might like to generate a larger test case, using random values.  sklearn has a tool, _datasets.make_regression_, that can create such test cases:

In [ ]:
ex2_n_samples = 50
ex2_x, ex2_y, ex2_coef = datasets.make_regression(
    n_samples = ex2_n_samples,
    n_features = 1,
    n_informative = 1,
    noise = 5,
    coef = True,
    random_state = 0,
)

* *n_samples* indicates the number of samples in the dataset to be generated.
* *n_features* indicates the number of inputs.  We are currently using 1, but later we will use more.
* *n_informative* indicates the number of those inputs that are 'important', that really affect the output values.  Any non-informative inputs do not influence the output, but simply confuse the regressor.
* Not shown here, *n_targets* indicates the number of output values.  By default, the value is 1.
* _noise_ indicates how much randomness should be added, how scattered are the output values.
* _coef_ indicates whether the coefficients of the generator should be returned.  These indicate the formula used to create the points, and ideally these are the values that the regression should compute.
* *random_state* is the seed value for the random number generator.  By specifying this value, then subsequent runs will generate the same sequence of random values.  Hence, what you see in your Notebook should match the values in my Notebook.

Let's see what got generated:

In [ ]:
plt.scatter(ex2_x, ex2_y)
plt.show()

This looks like a reasonable collection of data points.  We could ask for more randomness, or fewer or more points, and so on.

We can look to see what the return value looks like:

In [ ]:
ex2_x

We see that *ex2_x*, the inputs to the regression, is a 2-dimensional array.  The outer dimension or axis has one entry for every _row_ of our data, and the inner dimension, which is just 1 long, gives the number of input values.

We will see that the *ex2_y* value, the output, is just a 1-dimensional array.  If we asked for multiple targets, the output would also be a 2-dimensional array,

In [ ]:
ex2_y

*ex2_x* is a 2-dimensional array, but for some of the uses coming up, we want this as a 1-dimensional array.  There are a number of ways we can reconfigure an array.  Here we use the Numpy _reshape_ method.  We pass in an array, then a description of the shape we want: an array of integers giving the number of elements in each dimension.  We only want one dimension, and the number of elements we want 'in that dimension' is the size of the array.

In [ ]:
ex2_x = np.reshape(ex2_x, [ex2_x.size])
ex2_x

Now that we have the data in the format we want, we can perform the regression, generate the formula, and build the plot:

In [ ]:
ex2_z = np.polyfit(ex2_x, ex2_y, 1)
ex2_p = np.poly1d(ex2_z)
plt.scatter(ex2_x, ex2_y)
plt.plot(ex2_x, ex2_p(ex2_x), 'g-')
plt.show()

We can see the equation for this line:

In [ ]:
print(ex2_p)

We can also evaluate that line at any input value, as follows:

In [ ]:
ex2_p(1)

## Polynomial Linear Regression

Sometimes we have data that does not fit along a line, but rather along a curve.  Before we start, let's build a convenience function that will perform an analysis on our data:

In [ ]:
def poly_regress(x_data, y_data, degree) :
    z = np.polyfit(x_data, y_data, degree)
    p = np.poly1d(z)
    yhat_data = p(x_data)
    plt.scatter(x_data, y_data)
    plt.plot(x_data, yhat_data, 'b-')
    plt.show()
    print(f'Mean-square error: {np.sqrt(mean_squared_error(yhat_data, y_data)):.2f}')
    print('Formula:\n', p)

Consider the following dataset:

In [ ]:
x_a = range(1, 26)
y_a = [1.2, 0.8, 1, 0.9, 0.9, 1.3, 1, 1.7, 1.2, 1.8, 2, 1.8, 2, 3.2, 3.3, 4.2, 5.1, 5.2, 6, 7, 9, 10.1, 12, 14, 16]
poly_regress(x_a, y_a, 1)

As we can see, the straight line does a really poor job of fitting the set of data points.  Visually we see the mismatch, and the mean-square error is pretty bad.  _Note that this number is a lot smaller than the numbers we saw above, but that is not a surprise, since the numbers in this curve are a lot smaller._

With that evaluation, we were using a polynomial of degree 1.  Recall that the degree of a polynomial is the exponent of the largest power of _x_.  Consequently, our polynomial looked like this:

$$ y = \Omega_0 + \Omega_1 x^1 $$

Now let's try a polynomial of degree 2, which is a parabola.  The polynomial will look like this:

$$ y = \Omega_0 + \Omega_1 x^1 + \Omega_2 x^2 $$

In [ ]:
poly_regress(x_a, y_a, 2)

That is looking a lot better!  The line (curve) follows the points a lot better, and the error is a lot smaller.

Feeling quite happy about these results, we next try using a third-degree polynomial:

In [ ]:
poly_regress(x_a, y_a, 3)

That is looking even better!  The error has gone down even more, and the line is a much closer match to the points.

So let's try a fourth-degree polynomial:

In [ ]:
poly_regress(x_a, y_a, 4)

This is only a slight improvement over the third-degree polynomial, so perhaps we have gone far enough.  Also note that the leading coefficient is really small, hinting that this term is not strongly influencing the result.

These examples were following a simple curve.  Let's try a curve with an inflection point:

In [ ]:
x_b = range(1, 20)
y_b = [-6.5, -5.7, -3.5, -1.5, -0.8, .5, 1.2, 1.1, .8, 1.1, 1, 1.2, 2, 2, 3.2, 4.8, 6.4, 6.8, 8.6]
poly_regress(x_b, y_b, 1)

As before, not too good.  Degree 2:

In [ ]:
poly_regress(x_b, y_b, 2)

What!  The result is the same!  What happened?

OK, the result is not exactly the same, we can see in the formula that there is a small influence by the leading term.  But the error didn't improve.

The problem is, there is no parabola that fits this set of points (OK, technically, yes there is, but not really).

Let's move on to the third degree:

In [ ]:
poly_regress(x_b, y_b, 3)

Now we're doing pretty good.  The curve really seems to fit the points, and the error has gone way down.

4th degree:

In [ ]:
poly_regress(x_b, y_b, 4)

Again, not much of an improvement.  We should stick with a third-degree polynomial.

Let's try another curve, a little more complex:

In [ ]:
x_c = range(1, 19)
y_c = [8.8, 6.8, 4.7, 2.8, 2.1, 3, 3.7, 3.6, 2.5, 1.5, 0.7, 0.6, 1.0, 1.8, 2.9, 4.2, 6, 7.5]
poly_regress(x_c, y_c, 1)

Nope!  2nd-degree:

In [ ]:
poly_regress(x_c, y_c, 2)

Better, but needs some work!

3rd degree:

In [ ]:
poly_regress(x_c, y_c, 3)

This looks a lot like the last solution.  There is not a good fit for a 3rd degree polynomial.  4th?

In [ ]:
poly_regress(x_c, y_c, 4)

In [ ]:
poly_regress(x_c, y_c, 5)

In [ ]:
poly_regress(x_c, y_c, 6)

In [ ]:
poly_regress(x_c, y_c, 8)

In [ ]:
poly_regress(x_c, y_c, 10)

OK, we finally got a nice fit to the curve.  The problem is, if we use a polynomial of too high a degree, we are _overfitting_ the data, we are matching the variance of the data, rather than the general trend of the data.  Consider the following data:

In [ ]:
x_d = range(1, 19)
y_d = [12, 8, 9, 7, 6, 3, 2, 4, 3, 1, 0, 1.5, 2.7, .8, 2.2, 1, 1.2, 2]
poly_regress(x_d, y_d, 2)

In [ ]:
poly_regress(x_d, y_d, 16)

We probably would prefer the first solution, which generally matched the shape of the curve, rather than the second solution, which directly connected each of the points.  That second solution also captured and reproduced the 'noise'.

If we think back to our first cases, we noticed that for a while we were getting big reductions in the error measurement, but that after a point, the error measure didn't change by much.  This is a typical pattern.

We can pursue a new direction here: Why don't we graph the error as a function of the degree of the polynomial.
However, before we do, let's turn off the warnings.  Why?  In the following, we are going to ask polyfit to run over a wide range of degrees, and when we give really large degree, it prints a warning message about polyfit being poorly conditioned.

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

So now we implement, then run, a routine to test out various degrees of polynomial.

In [ ]:
def map_degree(x_data, y_data) :
    num_degrees = 30
    scores = []
    rng = range(1, num_degrees)
    for degree in rng:
        z = np.polyfit(x_data, y_data, degree)
        p = np.poly1d(z)
        yhat_data = p(x_data)
        scores.append(np.sqrt(mean_squared_error(yhat_data, y_data)))
    plt.plot(rng, scores, 'b-')


map_degree(x_a, y_a)

In [ ]:
map_degree(x_b, y_b)

In [ ]:
map_degree(x_c, y_c)

In [ ]:
map_degree(x_d, y_d)

These graphs show that there is a "knee" (or is it "elbow").  Before the elbow the error rate is rapidly dropping.  After that, it tapers off a bit.  Yes, the formula gets more accurate past that point, but that is because of overfitting.

## Building Larger Test Cases

In the previous section, we built toy examples, we hand-generated small lists of samples.  With a very limited number of samples, we couldn't do our normal train/test split of the data, so we could see how well the prediction worked on unseen examples.

So let's write a function that will generate polynomial test cases for us.  This function should take the following values:
* Coefficients for $ x^3, x^2, x^1 $ and $ x^0 $.
* A factor for the randomness of the values
* The number of points to generate
* The low and high limits for the _x_ coordinates
This function will return a DataFrame with two columns, the _x_ values and the corresponding _y_ values.

In [ ]:
def make_poly_regression_test(num_samples, x3coef, x2coef, x1coef, x0coef, random_factor, low_x, high_x) :
    x_vals = []
    y_vals = []
    for i in range(num_samples) :
        x = np.random.random() * (high_x - low_x) + low_x
        y0 = x * (x1coef + x * (x2coef + x * x3coef)) + x0coef
        y = y0 + np.random.randn() * random_factor
        x_vals.append(x)
        y_vals.append(y)
    return pd.DataFrame({'x': x_vals, 'y': y_vals}).sort_values(by = 'x', ascending = True)

In [ ]:
df = make_poly_regression_test(100, 0, 2, 3, 1, 10, 0, 20)
poly_regress(df.x, df.y, 2)

In [ ]:
df = make_poly_regression_test(50, 1, -2, 0, 2.5, 10, -5, 5)
poly_regress(df.x, df.y, 3)

Here we can see tests with a lot more data points, and we can see that the resulting formulas are fairly close to the actual generating formulas (at least the more significant terms, with the larger coefficients, are really close).

## Using Sklearn

Numpy's _polyfit_ works nicely for the cases we used above, but perhaps we should learn how to use the sklearn _LinearRegression_ method as well.

Here is one difference between the two:

* _polyfit_ is designed to fit polynomials of a specified degree.  So given just the _x_ inputs, _polyfit_ will also compute the $ x^2 $ values, $ x^3 $ values, and so on, as needed based on the degree.

* _LinearRegression_ is to handle multiple _features_ (input values), but only the single power of these features.

To use _LinearRegression_ to fit a polynomial curve, we've got a problem.

Luckily, there is a solution.  All we need to do, to fit a third-degree polynomial, is create two extra input columns in our DataFrame.  We started with the one column, containing the _x_ values.  We now need a second column filled with the square of the _x_ values, and a third column filled with the cube of the _x_ values.  _LinearRegression_ can then pick scaling factors for these three inputs, resulting in a polynomial fit!

To make these extra columns, sklearn has a method called _PolynomialFeatures_.  Using this method is a two step process:

1. Call _PolynomialFeatures_, passing in the desired degree of the polynomial.  This will return a generator.

2. Call the generator, passing in the array of _x_ values.  This will return a 2-dimensional array, where the first column is the _x_ values, the second column is the $ x^2 $ values, and so on.

Let's use this on our *x_b* data from above:

In [ ]:
x_b

This is a _range_ object, but we want an array.  Numpy has a method for creating an array from a range:

In [ ]:
np.array(x_b)

Getting back to PolynomialFeatures, let's create a generator that has a degree of 3 (for the *x_b* example, we want a polynomial of degree 3).  This function also takes a second parameter, *include_bias*, which we should always set to False (for linear regression, we don't want a 'constant term', the $x^0$ value):

In [ ]:
poly = PolynomialFeatures(degree = 3, include_bias = False)
poly

We can now run that generator to build our 'input array' for the linear regression, having columns for _x_, $x^2$, and $x^3$:

In [ ]:
x_b_arr = poly.fit_transform(np.array(x_b).reshape(-1,1))
x_b_arr

Now that we have these input columns, we can perform the LinearRegression:

In [ ]:
poly_reg_model = linear_model.LinearRegression()
poly_reg_model.fit(x_b_arr, y_b)
y_b_predicted = poly_reg_model.predict(x_b_arr)
plt.scatter(x_b, y_b)
plt.plot(x_b, y_b_predicted, 'b-')
plt.show()
print(f'Mean-square error: {np.sqrt(mean_squared_error(y_b_predicted, y_b)):.2f}')

Recalling the _polyfit_ solution, we see the similarities:

In [ ]:
poly_regress(x_b, y_b, 3)

Using sklearn was a bit more work, we needed to build the _PolynomialFeatures_, so there was a little more to keep track of.  But we will next see where the sklearn approach is quite useful.

## Multiple Linear Regressions (Multiple Inputs)

We will now build a dataset that has two inputs (features), but one of these does not strongly influence the output.  In other words, this input is some extraneous data that might simply confuse the regression.  This is a very common practice, as in real life, when we approach a dataset, we do not necessarily know which of the inputs really matter (at least for the particular output we are examining).

For the following, we build a regression dataset.  This will have two features, but only one is informative (influences the output):

In [ ]:
mlr_n_samples = 50
mlr_x, mlr_y, mlr_coef = datasets.make_regression(
    n_samples = mlr_n_samples,
    n_features = 2,
    n_informative = 1,
    noise = 2,
    coef = True,
    random_state = 0,
)

The *mlr_x* result is a 2-dimensional array, containing the two input columns, and *mlr_y* is the output column.  For convenience we convert the input data into a DataFrame:

In [ ]:
mlr_df = pd.DataFrame(mlr_x, columns = ['In1', 'In2'])
mlr_df

Let's perform a Linear Regression using both of the two input values.  After we have finished the regression, we will plot the results using the 'In1' data (the informational data) for the x-coordinate, and then we will also plot the results using the 'In2' data:

In [ ]:
model1 = linear_model.LinearRegression()
model1.fit(mlr_df, mlr_y)
yhat = model1.predict(mlr_df)
plt.scatter(mlr_df.In1, mlr_y)
plt.plot(mlr_df.In1, yhat, 'b-')
plt.show()

In [ ]:
plt.scatter(mlr_df.In2, mlr_y)
plt.plot(mlr_df.In2, yhat, 'r-')
plt.show()

Well, this was surprising.  First of all, the 'In2' data is all over the map, it does not correlate with the 'In1' data nor with the 'Output' data.  Also, it was not sorted, so the lines go all over the place!

We can do a quick sanity check: We can perform a linear regression using only In1, ignoring In2.  And then we can perform another linear regression using only In2, ignoring In1.  First, some code:

In [ ]:
def sk_regress(x_data, y_data, degree) :
    model = linear_model.LinearRegression()
    poly = PolynomialFeatures(degree = degree, include_bias = False)
    x_data_arr = poly.fit_transform(np.array(x_data).reshape(-1,1))
    model.fit(x_data_arr, y_data)
    yhat = model.predict(x_data_arr)
    plt.scatter(x_data, y_data)
    plt.plot(x_data, yhat, 'b-')
    plt.show()
    print(f'Mean-square error: {np.sqrt(mean_squared_error(yhat, y_data)):.2f}')

We now run a regression using only In1:

In [ ]:
sk_regress(mlr_df.In1, mlr_y, 1)

That was what we expected, the results look good.  Now perform the regression using only In2:

In [ ]:
sk_regress(mlr_df.In2, mlr_y, 1)

This is also what we would expect: there is no really great correlation between In2 and the output value.

So we have seen how the linear regression can have some extraneous input: the linear regression basically ignores this data.  Actually, the formula does consider this data, but the coefficient is really low.

Here is one other thing to consider: What if the output really _did_ depend upon both of the input values?  In a more general case, there may be many input values, some of which should be ignored, and some should be used.  But in the simpler case, let's build a problem where the output depends upon two input values (and not upon a third):

In [ ]:
mlr2_n_samples = 100
mlr2_x, mlr2_y, mlr2_coef = datasets.make_regression(
    n_samples = mlr2_n_samples,
    n_features = 3,
    n_informative = 2,
    noise = 2,
    coef = True,
    random_state = 0,
)
mlr2_df = pd.DataFrame(mlr2_x, columns = ['In1', 'In2', 'In3'])
model2 = linear_model.LinearRegression()
model2.fit(mlr2_df, mlr2_y)
yhat2 = model2.predict(mlr2_df)

OK, not so exciting (but theoretically all of the work has been done).

Let's view the scatterplots of each input vs the output:

In [ ]:
plt.scatter(mlr2_df.In1, mlr2_y)
plt.show()

In [ ]:
plt.scatter(mlr2_df.In2, mlr2_y)
plt.show()

In [ ]:
plt.scatter(mlr2_df.In3, mlr2_y)
plt.show()

So each of the plots is really spread out.  In1 is a little reasonable.  But the error rate will probably be quite high.  Let's see:

In [ ]:
print(f'Mean-square error: {np.sqrt(mean_squared_error(yhat2, mlr2_y)):.2f}')

Wow, that is really a small error!  Let's try doing independent regressions using each of the input values:

In [ ]:
sk_regress(mlr2_df.In1, mlr2_y, 1)

In [ ]:
sk_regress(mlr2_df.In2, mlr2_y, 1)

In [ ]:
sk_regress(mlr2_df.In3, mlr2_y, 1)

The results are much like we would expect just looking at the scatterplots.  The output is not correlated with any _single_ input value, but _is_ highly correlated with the combination of In1 and In2, and is independent of In3.  We can print the coefficients that were used by the generator to check our results:

In [ ]:
print(mlr2_coef)

## Sparse Data

Often in real-world data, there are uninformative variables in the data (features which don't markedly affect the output).  With these data in the dataset, the regression will try to use these values to help fit the samples.  Usually the simpler model is preferred.

For this section, we will use a database built-in to sklearn: The Boston housing market.  This dataset containes 13 features, and the question to be answered is: Can we predict the price of a new house given the values for these features?  The dataset also includes the median house price, so this column is the answer we will be attempting to predict.

As a preliminary step, we will load the database.  We can then find the keys for this object (the keys are the names of the attributes for this object):

In [ ]:
boston = datasets.load_boston()
print('Keys:', boston.keys())

Based on these names, this is what we might expect:
* _data_ is the list of inputs, or features.
* _target_ is the target value, in this case the median house price.
* *feature_names* is the column names for the input data.
* _DESCR_ is a description of the dataset
* The other two keys might not be as useful.

Let's print the description:

In [ ]:
print(boston.DESCR)

This description gave us a lot of information about the dataset.  Let's build a DataFrame holding the data, and let's add another column to the table, named 'price', which is the target value:

In [ ]:
df_boston = pd.DataFrame(boston.data, columns = boston.feature_names)
df_boston['price'] = boston.target

For our first visualization, let's plot a bar graph of the home prices.

In [ ]:
plt.hist(df_boston.price) 
plt.xlabel('price (\$1000s)')
plt.ylabel('count')
plt.show()

We have 13 features in the dataset.  Some of these will probably be more significantly correlated to the price, while others may be insignificant.  We can build quick linear regressions between some columns and the output, although as we saw above, _combinations of features_ may correlate more strongly that these features in isolation.

Seaborn has many interesting data visualization methods.  Check out the Seaboard documentation, they have a nice gallery showing representative diagrams.

The main visualization we will use now is the linear regression, using _lmplot_.  We pass in the dataset, indicate which columns to use on the two axes, and give the size of the plot.

For the first chart, let's compare _price_ with _LSTAT_ (the indication of lower income status):

In [ ]:
sns.lmplot(data=df_boston, x="price", y="LSTAT", height = 5.2, aspect = 2)
plt.show()

We can see that a linear equation does not fit the data too well, but that a curve would be better.  A quadratic might be a better fit.  While most places we would set the _degree_ of the polynomial, in Seaborn they used the name _order_:

In [ ]:
sns.lmplot(x="price", y="LSTAT", data=df_boston, order=2, height = 5.2, aspect = 2)
plt.show()

Another promising possibility is comparing the _price_ to _RM_, which is the number of rooms:

In [ ]:
sns.lmplot(x="price", y="RM", data=df_boston, height = 5.2, aspect = 2)
plt.show()

For that pair, linear seems like a fairly good match.

Next let's check the _CRIM_ column, an indication of the crime rate in the area:

In [ ]:
sns.lmplot(x="price", y="CRIM", data=df_boston, height = 5.2, aspect = 2)
plt.show()

In [ ]:
sns.lmplot(x="price", y="CRIM", data=df_boston, order = 2, height = 5.2, aspect = 2)
plt.show()

We've performed both a linear and quadratic regression.  Not sure which is best...

## Heat Map

We did some preliminary investigation by performing regressions between some interesting input columns and our output value.  These might give us some idea of which inputs to include and which to exclude when performing a multi-variable regression.

Another approach to determine which inputs to use is to perform a correlation between the columns.  A correlation matrix checks all pairs of columns in the dataset, and for each pair computes the correlation.  If there is a strong correlation between one of the inputs and the output, then that is an input of interest.

A DataFrame can compute the correlation matrix, and Seaborn can plot a Heat Map based on that matrix.  In the following, we select some of the columns, then produce the Heat Map.

In [ ]:
indexes = [0, 2, 4, 5, 6, 12]
df2 = pd.DataFrame(boston.data[:,indexes], columns = boston.feature_names[indexes])
df2['price'] = boston.target
corrmat = df2.corr()
sns.heatmap(corrmat, vmax = .8, square = True)
plt.show()

We are interested in the columns that are strongly correlated to the _price_.  Of course, each column is strongly correlated to itself!  But we can see that _price_ is strongly correlated to _RM_, the number of rooms.  _price_ is also strongly correlated to _LSTAT_, but this is a _negative correlation_: As _LSTAT_ goes up, _price_ tends to go down.  So in the Heat Map, we are looking for columns that are either really light colors or really dark colors.

## Scatter Plot

Another interesting visualization tool is a _scatter plot_.  For this, we first pick a small set of interesting columns.  We then build a new DataFrame containing just these columns (and our output _price_).  We then call the Pandas plotting *scatter_matrix* method:

In [ ]:
indexes = [5, 6, 12]
df2 = pd.DataFrame(boston.data[:,indexes], columns = boston.feature_names[indexes])
df2['price'] = boston.target
pd.plotting.scatter_matrix(df2, figsize=(12.0, 12.0))
plt.show()

On the diagonal of this diagram, we would be comparing each column with itself.  Instead, the program prints a bar chart of that column.  But in all of the non-diagonal cells, the diagram shows a scatter plot of the two columns (the row's column and the column's column!).

In the cells along the right edge, we get an overview of how each of the columns relates to the price.

However, in the other cells, we might find there is an interesting relationship between the two columns.

## LASSO

Up to now, we have only been looking at the _training data_.  Because of this, we have no idea how the _predictive_ calculations of the model perform.  In the past, we split the data into _training_ and _testing_ sets.  We trained with the training set, but predicted with the testing set.

We now do these steps:

In [ ]:
train_size = boston.data.shape[0]//2
x_train = boston.data[:train_size]
x_test = boston.data[train_size:]
y_train = boston.target[:train_size]
y_test = boston.target[train_size:]
print('Training and testing set sizes', x_train.shape, x_test.shape)

regr_boston = linear_model.LinearRegression()
regr_boston.fit(x_train, y_train) 
print('Coeff and intercept:', regr_boston.coef_, regr_boston.intercept_)

# Best possible score is 1.0, lower values are worse.
print('Training Score:', regr_boston.score(x_train, y_train) )
print('Testing Score:', regr_boston.score(x_test, y_test) )
print('Training MSE: ', np.mean((regr_boston.predict(x_train) - y_train)**2))
print('Testing MSE: ', np.mean((regr_boston.predict(x_test) - y_test)**2))

Well, it looks like our model is doing OK with the training data, but doing rather poorly with the testing data.  This is related to overfitting: our model is fairly well fit to the trained data.

Recall when we looked at the heat map, and even when we did some spot checks, some of the features were correlated to the output to some degree, but other features were not.  For example, _RM_ and _LSTAT_ were informative, but _AGE_ was not.

If we look at the array of coefficients, the model considered _all_ of the input features.  For every column, there is a non-zero coefficient.  Yes, some are small, but they all have an influence.

We can build a _sparse model_: we only consider some of the features, and the remaining features are ignored.  To do this we can create a _LASSO_ regressor (least absolute shrinkage and selection operator), which forces some of the coefficients to zero.

In [ ]:
regr_lasso = linear_model.Lasso(alpha=.3)
regr_lasso.fit(x_train, y_train) 
print('Coeff and intercept:', regr_lasso.coef_,  regr_lasso.intercept_)

print('Training Score:', regr_lasso.score(x_train, y_train))
print('Testing Score:', regr_lasso.score(x_test, y_test))

print('Training MSE: ', np.mean((regr_lasso.predict(x_train) - y_train)**2))
print('Testing MSE: ', np.mean((regr_lasso.predict(x_test) - y_test)**2))

Because of the LASSO, the coefficient array has four entries that are zero: these features are ignored in the regression.  The factors dropped are _CRIM_ (the crime rate), _INDUS_ (percentage of industrial buildings), _CHAS_ (being located on the Charles River, and _NOX_ (nitrous oxide).  The testing score went from -2.24 to 0.5, and the MSE went from 303 to 47.  This shows that the four factors were not important for the prediction, and in fact they confused the regressor.

We can get a report ranking the features in order of importance, from least to greatest, as follows:

In [ ]:
ind = np.argsort(np.abs(regr_lasso.coef_))
print('Order variable (from less to more important):', boston.feature_names[ind])

## Select K Best

Another approach to finding the most important features is the _SelectKBest_ function from sklearn.

This function takes an input parameter, _k_, which indicates the number of features we wish to include.  The function's output is an array listing all of the features, each with a value of True (keep this one) or False (ignore this one).

In [ ]:
import sklearn.feature_selection as fs 
selector = fs.SelectKBest(score_func = fs.f_regression, k = 5)
selector.fit_transform(x_train, y_train)
z = list(zip(selector.get_support(), boston.feature_names))
print('Selected features:', z)

## Conclusion

We've covered a lot of stuff.

Recall that linear regression is the most-used machine learning algorithm.

But there are several additional machine learning algorithms that we will be studying.